In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [2]:
from config_paper import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-27'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'400000'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.25

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [ ]:
for minu in range(500) :
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-20', to=startdate,).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:

                try:
                    openPosition = api.get_position(symbol)
                    print(openPosition.qty)
                except:
                    cashBalance = account.cash

                    targetPositionSize = int(float(cashBalance) / (data['close'][-1] / positionSizing)) # Calculates required position size

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print(returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 75 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position  SMA50
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print(returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)

Order({   'asset_class': 'us_equity',
    'asset_id': 'a75fec50-9c3a-4806-ad72-a4f27e22db28',
    'canceled_at': None,
    'client_order_id': 'ce9f3146-9c35-4c1b-9cc9-6aa4bfc1c273',
    'created_at': '2020-04-27T19:57:26.298106Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '580c0746-9781-4326-a318-5408c141881e',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '600',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-27T19:57:26.282591Z',
    'symbol': 'ADS',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-27T19:57:26.298106Z'})
BRKB
BFB
Order({   'asset_class': 'us_equity',
    'asset_id': '94cd9972-28f5-4445-ab72-e326662b0b9d',
    'canceled_at': None,
    'client_or

FIS
606
Order({   'asset_class': 'us_equity',
    'asset_id': 'ef9adcc6-6bce-452b-af5f-4a87e7f19131',
    'canceled_at': None,
    'client_order_id': '976f0f68-a7fd-4c68-aa30-0a58d67bf65b',
    'created_at': '2020-04-27T20:02:56.525545Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '204678be-f926-4647-a2a3-2c9222ffdc0e',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '162',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-27T20:02:56.451571Z',
    'symbol': 'JNJ',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-27T20:02:56.525545Z'})
LHX
MKTK
NFLX
RTN
